In [1]:
import torch
import numpy as np

import mdtraj as md
import nglview as nv

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

from tqdm import tqdm

n_particles = 22
n_dimensions = 3
dim = n_particles * n_dimensions

# file_name = "0414_100211"
file_name = "debug"
state = "c5"
scaling = 1.4936519791


# Visualize samples

In [2]:
latent = torch.load(f"../res/{file_name}/result/latent-{state}.pt")
samples = torch.load(f"../res/{file_name}/result/samples-{state}.pt")
dlogp = torch.load(f"../res/{file_name}/result/dlogp-{state}.pt")
ad2_topology = md.load("../data/AD2/c5-tbg.pdb").topology
samples.shape

torch.Size([100, 66])

In [3]:
samples_view = samples / scaling
traj = md.Trajectory(
	xyz=samples_view.cpu().detach().numpy().reshape(-1, n_particles, n_dimensions),
	topology=ad2_topology
)

In [4]:
view = nv.show_mdtraj(traj)
view

NGLWidget(max_frame=99)